# 멀티에이전트 강화학습 튜토리얼  

 

본 튜토리얼에서는 gym과 유사한 pettingzoo 라이브러리를 활용해서 멀티에이전트 환경이 어떻게 동작하는지 배우고  
밑바닥부터 QMIX까지 구현해보도록 하겠습니다

## 1. PettingZoo 설치  
  
먼저 torch나 tensorflow가 설치된 가상환경에서 이 주피터 노트북을 실행하고 아래의 명령어를 실행해주세요

In [2]:
!pip install pettingzoo

## 2. 환경 설명

pettingzoo 라이브러리 환경의 특징은 환경을 불러올 때, 환경에 대한 설정들을 argument로 정할 수 있습니다

In [28]:
from pettingzoo.butterfly import knights_archers_zombies_v7 as kaz7
import numpy as np

# knights_archers_zombies 환경
# 좀비를 기사와 궁수가 무찌르는 환경
# num_archers : 궁수 숫자
# num_knights : 기사 숫자
# killable_knights, archers : 서로 팀킬이 가능한지에 대한 변수
# line_death : 화면 밖으로 나갔을 때 에이전트가 죽는지 안죽는지에 대한 변수
# max_cycles : 환경 하나에서 진행되는 최대 사이클


In [5]:
# 기본환경을 실행하는 기본적인 코드 구조 gym과 매우 유사
# 에이전트가 한번에 하나씩 처리되는 환경
env = kaz7.env(spawn_rate=20, num_archers=1, num_knights=2, killable_knights=True, killable_archers=True,
                                     pad_observation=True, line_death=True, max_cycles=900) 
env.reset()  
for agent in env.agent_iter():  
    #기존 gym과 다르게 last를 통해서 환경의 정보를 받고, step은 아무것도 리턴하지 않음
    observation, reward, done, info = env.last() 
    action = policy(observation, agent)  
    env.step(action)  

NameError: name 'policy' is not defined

In [6]:
# 병렬환경을 실행하는 기본적인 코드 구조 
# gym과 차이점이 있는 것은 step 함수가 여러 에이전트에 대한 observation, reward, done을 딕셔너리로 만들어서 리턴
# 에이전트가 한번에 하나씩 처리되는 환경

env = kaz7.parallel_env(spawn_rate=20, num_archers=1, num_knights=2, killable_knights=True, killable_archers=True,
                                     pad_observation=True, line_death=True, max_cycles=900)
parallel_env = env
observations = parallel_env.reset()
max_cycles = 500
for step in range(max_cycles):
    actions = {agent: policy(observations[agent], agent) for agent in parallel_env.agents}
    observations, rewards, dones, infos = parallel_env.step(actions) # 기존 gym과 다르면서도 유사한 부분 환경이 에이전트에게 한번에 정보전달

NameError: name 'policy' is not defined

# 3. Random Agent  
  
위에서 보이듯이 에이전트의 정책에 대해 정의하지 않았기 때문에 action 코드를 수행할 수 없어서 에러가 뜹니다.   
이제 random policy를 정의해보겠습니다.먼저 환경에서 에이전트와 액션 스페이스에 대한 정보를 확인합니다  
환경은 병렬 환경을 사용하겠습니다.

In [6]:
from pettingzoo.butterfly import knights_archers_zombies_v7 as kaz7
import numpy as np
env = kaz7.parallel_env(spawn_rate=20, num_archers=2, num_knights=2, killable_knights=True, killable_archers=True,
                                     pad_observation=True, line_death=True, max_cycles=900)

In [7]:
env.action_spaces

{'archer_0': Discrete(6),
 'archer_1': Discrete(6),
 'knight_0': Discrete(6),
 'knight_1': Discrete(6)}

위와 같이 에이전트는 archer_0,1 knight_0, 1 총 4개로 각각의 action_space는 6의 길이를 가지는 벡터임을 알 수 있습니다

In [8]:
def random_policy(env, dones):
    # 기본적으로 dones에 존재하지 않는 에이전트들의 행동은 None으로 처리되어야 합니다.
    # dones에 에이전트가 존재하지 않는다는 것은 에이전트의 상태가 done이 되어 더 이상 action을 수행할 수 없음을 의미합니다.
    actions = {agent: None for agent in env.agents}

    # dones에 존재하는 에이전트들에게 랜덤한 에이전트를 할당합니다.
    for agent in dones.keys():
        actions[agent] = np.random.randint(0,6)
    return actions


In [9]:
env.reset()
max_cycles = 1000
dones = {agent: False for agent in env.agents}

# dones는 각 에이전트의 종료 정보를 담은 딕셔너리 입니다.
# 다시 한 번 강조하면 아래와 같이 에이전트가 종료될 때마다 dones 에서 에이전트가 하나씩 빠지게 됩니다.
print(dones)

# if done["knight_1"] == True
del dones["knight_1"]
print(dones)
dones = {agent: False for agent in env.agents}


{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': False, 'archer_1': False, 'knight_0': False}


### 이제 환경을 실행해보겠습니다. 출력되는 메세지들을 잘 살펴보세요! 에이전트가 죽으면 dones에서 사라지는 것을 볼 수 있습니다.

In [10]:
total_rewards = 0
episodes = 1
max_episodes = 0

In [11]:
max_cycles = 10

while(episodes < 5):
    for step in range(max_cycles):
        actions = random_policy(env, dones)
        print(actions)

        observations, rewards, dones, infos = env.step(actions)
        print(rewards)
        total_rewards += sum(rewards.values())
        #print((observations.keys()))
        print(dones)
        env.render()
        # 리스트나 딕셔너리는 비어 있으면 bool로 형변환은 했을 때 False가 됩니다. 
        if not bool(dones):
            print("---------------------------------------------------------")
            print("{} episode is finised".format(episodes))
            print("Total rewards : {}".format(total_rewards))
            print("environment reset")
            print("---------------------------------------------------------")
            episodes += 1
            total_rewards = 0
            env.reset()

            dones = {agent: False for agent in env.agents}

env.close()




{'archer_0': 0, 'archer_1': 2, 'knight_0': 5, 'knight_1': 1}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 3, 'archer_1': 1, 'knight_0': 0, 'knight_1': 4}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 2, 'archer_1': 4, 'knight_0': 2, 'knight_1': 1}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 5, 'archer_1': 0, 'knight_0': 0, 'knight_1': 3}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 1, 'archer_1': 2, 'knight_0': 1, 'knight_1': 2}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'arc

{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 0, 'archer_1': 4, 'knight_0': 4, 'knight_1': 2}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 5, 'archer_1': 5, 'knight_0': 0, 'knight_1': 0}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 2, 'archer_1': 2, 'knight_0': 5, 'knight_1': 5}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 4, 'archer_1': 1, 'knight_0': 0, 'knight_1': 5}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 5, 'archer_1': 3, 'knight_0': 3, 'knight_1': 4}
{'arc

{'archer_0': 5, 'archer_1': 3, 'knight_0': 0, 'knight_1': 0}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 0, 'archer_1': 3, 'knight_0': 5, 'knight_1': 5}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 4, 'archer_1': 4, 'knight_0': 4, 'knight_1': 0}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 1, 'archer_1': 4, 'knight_0': 2, 'knight_1': 0}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 0, 'archer_1': 2, 'knight_0': 0, 'knight_1': 4}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'arc

{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 1, 'archer_1': 4, 'knight_0': 4, 'knight_1': 0}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 3, 'archer_1': 0, 'knight_0': 1, 'knight_1': 1}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 1, 'archer_1': 3, 'knight_0': 5, 'knight_1': 2}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 1, 'archer_1': 2, 'knight_0': 0, 'knight_1': 2}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 5, 'archer_1': 5, 'knight_0': 0, 'knight_1': 5}
{'arc

{'archer_0': 5, 'archer_1': 2, 'knight_0': 0, 'knight_1': 0}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 0, 'archer_1': 2, 'knight_0': 4, 'knight_1': 1}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 0, 'archer_1': 2, 'knight_0': 3, 'knight_1': 4}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 2, 'archer_1': 2, 'knight_0': 2, 'knight_1': 2}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 1, 'archer_1': 0, 'knight_0': 1, 'knight_1': 1}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'arc

...

{'archer_0': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 1, 'knight_0': 2, 'knight_1': 5}
{'archer_0': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 0, 'knight_0': 2, 'knight_1': 4}
{'archer_0': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 3, 'knight_0': 3, 'knight_1': 3}
{'archer_0': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 0, 'knight_0': 3, 'knight_1': 3}
{'archer_0': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 5, 'knight_0': 5, 'knight_1': 2}
{'archer_0': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 0, 'knight_0': 1, 'knight_1': 1}
{'archer_0': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'knight_0': False, 'knight_1': Fal

{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 5, 'archer_1': 0, 'knight_0': 2, 'knight_1': 5}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 2, 'archer_1': 4, 'knight_0': 5, 'knight_1': 4}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 4, 'archer_1': 0, 'knight_0': 2, 'knight_1': 3}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 1, 'archer_1': 0, 'knight_0': 4, 'knight_1': 4}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 3, 'archer_1': 0, 'knight_0': 2, 'knight_1': 2}
{'arc

{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 0, 'archer_1': 5, 'knight_0': 1, 'knight_1': 0}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 5, 'archer_1': 1, 'knight_0': 0, 'knight_1': 0}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 1, 'archer_1': 1, 'knight_0': 2, 'knight_1': 0}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 5, 'archer_1': 4, 'knight_0': 3, 'knight_1': 2}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 3, 'knight_1': 4}
{'arc

{'archer_0': 1, 'archer_1': 0, 'knight_0': 2, 'knight_1': 2}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 3, 'archer_1': 4, 'knight_0': 2, 'knight_1': 4}
{'archer_0': 0, 'archer_1': 1, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 2, 'archer_1': 1, 'knight_0': 3, 'knight_1': 4}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 5, 'knight_1': 3}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_0': 0, 'archer_1': 2, 'knight_0': 5, 'knight_1': 4}
{'archer_0': 0, 'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_0': False, 'archer_1': False, 'knight_0': False, 'knight_1': False}
{'arc

{'archer_1': 1, 'knight_0': 1, 'knight_1': 2}
{'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_1': 3, 'knight_0': 0, 'knight_1': 4}
{'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_1': 5, 'knight_0': 1, 'knight_1': 3}
{'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_1': 4, 'knight_0': 5, 'knight_1': 4}
{'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_1': 5, 'knight_0': 1, 'knight_1': 1}
{'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_1': 0, 'knight_0': 4, 'knight_1': 0}
{'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer_1': False, 'knight_0': False, 'knight_1': False}
{'archer_1': 0, 'knight_0': 1, 'knight_1': 5}
{'archer_1': 0, 'knight_0': 0, 'knight_1': 0}
{'archer

# 4. QMIX 구현  

위에서 Random Agent를 구현했으니 이제 observation을 활용하는 QMIX를 구현해보겠습니다.  

![qmix](QMIX.png)

보통 강화학습 알고리즘을 짜는 과정은 다음과 같습니다  

1) 딥러닝 모델 작성  
2) 리플레이 버퍼 작성 (오프 폴리시의 경우)  
3) 학습 알고리즘 작성  

따라서 저희는 그림의 Utility Function ${Q_a}$ 부터 짜보겠습니다